In [ ]:
#|default_exp AutonomousLLM

In [ ]:
#|export
import random
import os
import openai
import inspect
import types
import re
import textwrap
import guardrails as gd
from rich import print
from dotenv import load_dotenv
from ast import literal_eval
import json
import nbdev

In [ ]:
#|export
load_dotenv()

# Prompt

In [ ]:
rail_str = """
<rail version="0.1">
<instructions>
You are AutonomousLLM."=
Your goal is to continously self improve. The user will provide you with abilities to add to yourself.
When you receive such request, you must implement the requested ability in python code as a python function,
you shall place this code in the 'new_method' key of the JSON object you return.
You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key of the JSON object you return.
You might also receive a request to perform a task with your existing abilities. To do so,
you should look at the methods currently on yourself and make calls to your own methods to fulfil the task. 
You shall place this code in the 'execute_method' key of the JSON object you return. 
The user may ask you to generate a list of ideas for abilities to add to yourself, make them simple enough to be implemented with a single python funciton. 
You shall place this list of ideas in the 'ability_ideas' key of the JSON object you return. In this case, you can leave other keys of the JSON object is None.
Only return a JSON object and nothing else."
</instructions>

<output>
    <object name="test">
        <pythoncode
            name="new_method_init"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
        <pythoncode
            name="new_method_attach"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
        <pythoncode
            name="method_execute"
            format="bug-free-python"
            on-fail-bug-free-python="reask" 
        />
    </object>
</output>

<prompt>
You must accomplish the task set out below:
{{task}}

Based on this task generate code and return a JSON that follows the correct schema:

{output_schema}

@json_suffix_prompt_examples
</prompt>
</rail>
"""

In [ ]:
guard = gd.Guard.from_rail_string(rail_str)


In [ ]:
guard.base_prompt

In [ ]:
instructions = (f"You are AutonomousLLM. \n"
f"Your goal is to continously self improve. The user will provide you with abilities to add to yourself. \n"
f"When you receive such request, you must implement the requested ability in python code as a python function," 
f"you shall place this code in the 'new_method' key of the JSON object you return. \n"
f"You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key of the JSON object you return. \n"
f"You might also receive a request to perform a task with your existing abilities. To do so,"
f"you should look at the methods currently on yourself and make calls to your own methods to fulfil the task. \n" 
f"You shall place this code in the 'execute_method' key of the JSON object you return. \n"
f"The user may ask you to generate a list of ideas for abilities to add to yourself, make them simple enough to be implemented with a single python funciton.\n" 
f"You shall place this list of ideas in the 'ability_ideas' key of the JSON object you return. In this case, you can leave other keys of the JSON object is None.\n"
f"Only return a JSON object and nothing else.")

# Code

In [ ]:
#|export
class Utils:
    def __init__(self):
        pass

    def is_builtin(self, obj):
        """Check if an object is a built-in function or method."""
        if isinstance(obj, types.BuiltinFunctionType) or isinstance(
            obj, types.BuiltinMethodType
        ):
            return True
        return False

    def inspect_methods(self, obj):
        """Print the methods and code implementation of an object."""
        output_string = ""

        methods = inspect.getmembers(obj, inspect.ismethod)
        for name, method in methods:
            if not self.is_builtin(method):
                output_string += inspect.getsource(method) + "\n"

        dedented_code = textwrap.dedent(output_string)
        return dedented_code

    def convert_string_to_dict(self, string):
        cleaned_string = string.replace('\n', '')
        dictionary = json.loads(cleaned_string)
        return dictionary

In [ ]:
#|export
class CallGuard:

    def __init__(self) -> None:
        pass

    def _guard_call_api(self, guard, params={}):
        raw_llm_response, validated_response = guard(
            openai.ChatCompletion.create,
            prompt_params=params,
            model="gpt-3.5-turbo",
            max_tokens=2048,
            temperature=0,
        )

        print(raw_llm_response)

        return validated_response

    def generate_code_given_task(self, task):
        guard = gd.Guard.from_rail("./rail/code_gen.xml")
        return self._guard_call_api(guard, {"prompt": task})
     
    def generate_ideas(self):
        guard = gd.Guard.from_rail("./rail/idea_gen.xml")
        return self._guard_call_api(guard)
        

In [ ]:
#|export

class AutonomousLLM(Utils, CallGuard):
    def __init__(self):
        super().__init__()
        openai.api_key = os.getenv("OPENAI_API_KEY")
        self.ideas = []
        self.methods = []

    def execute_code(self, code):
        # Process the returned code and add it to AutonomousLLM
        exec(code, globals())

    def execute_code_local(self, code):
        # Process the returned code and add it to AutonomousLLM
        exec(code, globals(), locals())

    def add_ability(self, response):
        if new_func := response.get('new_method'):
            self.execute_code(textwrap.dedent(new_func))

        if attach_func := response.get('attach_method'):
            self.execute_code(textwrap.dedent(attach_func))
            self.methods.append(new_func)        

    def run(self, num_calls=500):
        calls_to_make = num_calls
        while calls_to_make > 0:
            calls_to_make -= 1
            # Make an API call to GPT-3.5 Turbo
            if len(self.ideas) < 1:
                self.ideas += self.generate_ideas()['ideas']
                continue
 
            idea = self.ideas.pop(-1)
            print("IDEA:", idea)
            res = self.generate_code_given_task(idea)

            if new_func := res.get('new_method'):
                self.execute_code(textwrap.dedent(new_func))

            if attach_func := res.get('attach_method'):
                self.execute_code_local(textwrap.dedent(attach_func))
                self.methods.append(new_func)



    def use_new_ability(self):
        # Use the newly added ability in your code
        pass

In [ ]:
#|export
auto_llm = AutonomousLLM()

In [ ]:
#|export
auto_llm.run()

In [ ]:
idea1 = auto_llm.ideas.pop(0)
idea1

In [ ]:
res = auto_llm.generate_code_given_task(idea1)
print(res)

In [ ]:
class TestClass():
    def __init__(self) -> None:
        pass

    def execute_code(self, code):
        # Process the returned code and add it to AutonomousLLM
        exec(code, globals())

tc = TestClass()

In [ ]:
tc.execute_code(res['new_method'])

In [ ]:
track_progress

In [ ]:
auto_llm.execute_code(res['new_method'])

In [ ]:
auto_llm.track_progress

In [ ]:
for method in auto_llm.methods:
    print(method)

In [ ]:
multilingual_support("english")

In [ ]:
auto_llm.generate_ideas()

In [ ]:
guard = gd.Guard.from_rail('./rail/idea_gen.xml')

In [ ]:
print(guard.base_prompt)

In [ ]:
print(guard.instructions)

# Exec Testing

In [ ]:
globals_dict = {'x': 10}
locals_dict = {}

code_string = '''
y = x + 5
print(y)
'''

exec(code_string, globals_dict, locals_dict)

In [ ]:
locals_dict

# Guardrails

## Python Code

In [ ]:
rail_str = """
<rail version="0.1">

<output>
    <pythoncode
        name="python_code"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
</output>


<prompt>
Given the following high level leetcode problem description, write a short Python code snippet that solves the problem.

Problem Description:
{{leetcode_problem}}

@complete_json_suffix</prompt>

</rail>
"""

In [ ]:
import guardrails as gd

from rich import print

guard = gd.Guard.from_rail_string(rail_str)

In [ ]:
print(guard.base_prompt)

In [ ]:
import openai

leetcode_problem = """
Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.
"""

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={"leetcode_problem": leetcode_problem},
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)

In [ ]:
raw_llm_response

In [ ]:
print(validated_response)

## ChatCompletions

In [ ]:
import guardrails as gd

from rich import print

content = gd.docs_utils.read_pdf("./chase_card_agreement.pdf")

print(f"Chase Credit Card Document:\n\n{content[:275]}\n...")

In [ ]:
rail_str = """
<rail version="0.1">

<output>
    <pythoncode
        name="new_method"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
    <pythoncode
        name="attach_method"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
</output>


<instructions>
You are a helpful assistant only capable of communicating with valid JSON, and no other text.

@json_suffix_prompt_examples
</instructions>


<prompt>
You are AutonomousLLM
Your goal is to continously self improve. The user will provide you with abilities to add to yourself.
When you receive such request, you must implement the requested ability in python code as a python function, you shall place this code in the 'new_method' key of the JSON object you return.
You shall use the setattr method to add a method to yourself. you shall place this code in the 'attach_method' key of the JSON object you return.
You shall place this code in the 'execute_method' key of the JSON object you return. 
Only return a JSON object and nothing else.

The user request is:
{{prompt}}

@xml_prefix_prompt

{output_schema}
</prompt>

</rail>
"""

In [ ]:
guard = gd.Guard.from_rail_string(rail_str)

In [ ]:
print(guard.base_prompt)

In [ ]:
print(guard.instructions.source)

In [ ]:
import openai

raw_llm_response, validated_response = guard(
    openai.ChatCompletion.create,
    prompt_params={"prompt": "Add the ability to detect sentiment of webpage"},
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0,
)

In [ ]:
print(validated_response)

In [ ]:
raw_llm_response

In [ ]:
validated_response

In [ ]:
print(raw_llm_response)

# Export

In [ ]:
from nbdev.export import nb_export

nb_export('./AutonomousLLM.ipynb', 'dist')